In [19]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import ChatPromptTemplate
load_dotenv()
#_ = load_dotenv(find_dotenv())
#groq_api_key  = os.environ.get["GROQ_API_KEY"]


True

In [12]:
def completion(prompt, model="llama-3.3-70b-versatile"):
    message=[{"role": "user", "content": prompt}]
    response = ChatGroq(model=model, temperature=0.0).invoke(message)
    return response.content

In [14]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [15]:
style = """American English \
in a calm and respectful tone
"""
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

In [17]:
response = completion(prompt)

In [18]:
response

"I'm having a bit of a frustrating issue with my blender. The lid came off unexpectedly and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I would greatly appreciate your assistance with this problem as soon as possible."

In [20]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [21]:
prompt_templete = ChatPromptTemplate.from_template(template_string)

In [25]:
prompt_templete.messages[0].prompt.input_variables

['style', 'text']

In [26]:
customer_style = """American English \
in a calm and respectful tone
"""

In [27]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [28]:
customer_messages = prompt_templete.format_messages(
                    style=customer_style,
                    text=customer_email)

In [30]:
# Call the LLM to translate to the style of the customer message
chatmodel = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.0)

customer_response = chatmodel(customer_messages)

C:\Users\Azeem Khan\AppData\Local\Temp\ipykernel_3304\1408907614.py:4: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chatmodel(customer_messages)


In [31]:
print(customer_response.content)

Here's the translation of the text into American English in a calm and respectful tone:

``` 
I'm really frustrated that my blender lid came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, please.
```


In [33]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [32]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [34]:
service_messages = prompt_templete.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [35]:
service_response = chatmodel(service_messages)
print(service_response.content)

```Ahoy, valued customer! Yer lookin' fer some help with yer kitchen contraption, eh? Alright then, matey, let's set sail fer a solution. Unfortunately, it seems ye didn't follow the proper treasure map... er, instructions fer yer blender. It appears ye forgot to secure the lid before startin' the blighter, and that's led to a right ol' mess! Now, I know it be frustratin', but our warranty doesn't cover cleanin' expenses in cases o' misuse, savvy? So, it seems ye be out o' luck this time, me hearty. But never fear, we be willin' to help ye find a way to get yer kitchen shipshape again. Fair winds, and may yer future blendin' adventures be lid-tastic!```


In [36]:
# NEW WORKING  WITH PARSER #

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [40]:
prom_tem = ChatPromptTemplate.from_template(review_template)
print(prom_tem)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [48]:
get_temp = prom_tem.format_messages(text= customer_review)
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.0)
response = llm.invoke(get_temp)

In [49]:
print(response.content)

```json
{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [50]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [51]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [52]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [53]:
format_instructions = output_parser.get_format_instructions()

In [54]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [56]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [57]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [58]:
response = llm(messages)

In [59]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features"
}
```


In [60]:
output_dict = output_parser.parse(response.content)